In [1]:
# Importing the necessary libraries
import pandas as pd
import re
from collections import defaultdict
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import numpy as np
import gensim
import gensim.downloader as api
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Import code from other files
from model import model as m

In [3]:
# Fit the model

# Get data and model
data, meta_data, valid_words = m.get_data()
w2v_model = m.get_model()
# Preprocess the data
data, word2vec = m.w2v_preprocessing(data, valid_words, w2v_model)
# Get the vectorized wordcount data
X_sparse, word_to_index, movie_id_to_index = m.get_vectorized_data(data)
# Get the tf-idf matrix
tf_idf = m.compute_tf_idf(X_sparse)
print(tf_idf.shape)


Number of words in the model:  39291
Number of words not in the model (should be 0 now):  0


/Users/antoine/anaconda3/envs/gensim_env/lib/python3.10/site-packages/scipy/sparse/_base.py:665: RuntimeWarning: invalid value encountered in divide
  return np.true_divide(self.todense(), other)


(42303, 39290)


In [4]:
# Get the movie vectors
movie_vectors = m.compute_movie_vectors(tf_idf, w2v_model, word_to_index, movie_id_to_index, data)

In [11]:
# clustering the movies

df_plot = pd.read_csv('../data/MovieSummaries/plot_summaries.txt', sep="\t", header=None)
df_plot.columns = ["Movie ID", "Plot_summary"]

kmeans = KMeans(n_clusters=5, random_state=0).fit(movie_vectors)

# Get the 5 movies closest from each cluster center
for i in range(15):
    print("Cluster", i+1)
    # Print the 5 closest words to the cluster center
    print("Closest words : ", [w2v_model.similar_by_vector(kmeans.cluster_centers_[i])[j][0] for j in range(10)])
    # For each cluster, print the movie descriptions to see if the clustering makes sense
    closest_movies_ids = data.iloc[np.argsort(np.linalg.norm(movie_vectors - kmeans.cluster_centers_[i], axis=1))[:5]]["Wikipedia_movie_ID"].values
    for j in range(5):
        print()
        print("Movie :", j, df_plot[df_plot["Movie ID"].isin(closest_movies_ids)]["Plot_summary"].values[j])
        print()

Cluster 1
Closest words :  ['pretending', 'pretend', 'thinks', 'telling', 'knowing', 'two-timing', 'something', 'terrified', 'afraid', 'indeed']

Movie : 0 The three meet each other for the first time at Mumbai's airport and decide to find a place together. In the course of the movie, Krishna finds out that, in order to break into the local film industry, he must first win the  patronage of Don Mastana , a godfather of the Mumbai underworld who's also a film producer. Mastana is a violent man who thinks nothing of impaling a lizard with a knife or shattering the skull of a fellow crime boss for making a pass at his girlfriend. Ricardo, the serious-looking Australian, finds out the sad fate of his brother, but also manages to fall in love with Mastana's spunky moll Dolly , igniting further flames. Xerxes, who's a Parsi, is led to embrace his latent homosexuality by their gay landlord .


Movie : 1 Chevy Chase once again plays the reporter Irwin M. "Fletch" Fletcher, who learns that he h

IndexError: index 5 is out of bounds for axis 0 with size 5